################ IMPORTING THE REQUIRED LIBRARIES

In [13]:
import numpy as np
import soundfile
import os, glob
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd
import librosa
from sklearn import preprocessing 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import GridSearchCV


############## EMOTIONS INCLUDED IN THE DATASET

In [14]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

############## EXTRACTING FEATURES FROM THE AUDIO SIGNAL USING LIBROSA

In [15]:
def extract_feature(file_name, mfcc, chroma,spectral_centroid,spectral_bandwidth,
                    spectral_rolloff,spectral_contrast,rms,spectral_flatness):
    with soundfile.SoundFile(file_name) as sound_file:
        X = sound_file.read(dtype="float32")
        sample_rate=sound_file.samplerate
        if chroma:
            stft=np.abs(librosa.stft(X))
        result=np.array([])
        if mfcc:
            mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
            result=np.hstack((result, mfccs))
            
            
        if chroma:
            chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
            result=np.hstack((result, chroma))             
           
            
        if spectral_centroid:
            spectral_centroid=np.mean(librosa.feature.spectral_centroid(y=X, sr=sample_rate))
            result=np.hstack((result, spectral_centroid)) 
        
        if spectral_bandwidth:
           spectral_bandwidth=np.mean(librosa.feature.spectral_bandwidth(y=X, sr=sample_rate).T)
#           print(spectral_bandwidth)
           result=np.hstack((result, spectral_bandwidth)) 
           
        if spectral_rolloff:
           spectral_rolloff=np.mean(librosa.feature.spectral_rolloff(y=X, sr=sample_rate).T)
#           print(spectral_rolloff)
           result=np.hstack((result, spectral_rolloff))
        
        if spectral_contrast:
           spectral_contrast=np.mean(librosa.feature.spectral_contrast(y=X, sr=sample_rate))
           result=np.hstack((result, spectral_contrast))
           
        if rms:
           rms=np.mean(librosa.feature.rms(y=X).T,axis=0)
           result=np.hstack((result, rms))
           
        if spectral_flatness:
           spectral_flatness=np.mean(librosa.feature.spectral_flatness(y=X))
           result=np.hstack((result, spectral_flatness))
        
        return result

############## LOADING THE DATASET AND EXTRACTING ALL THE FEATURES FROM IT 

In [16]:
def load_data(test_size=0.2):
    x,y=[],[]
    for file in glob.glob(r"F:\speech_project\speech-emotion-recognition-ravdess-data\Actor_*\*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                                spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
        x.append(feature)
        y.append(emotion)
    return x,y


In [18]:
X,y=load_data()
X=pd.DataFrame(X)
y=pd.DataFrame(y)
data=pd.concat([X,y],axis="columns")
data=data.sample(frac=1).reset_index(drop=True)
X=data.iloc[:,0:58]
y=data.iloc[:,58]

In [19]:
le = preprocessing.LabelEncoder()
y=le.fit_transform(y)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(X, y, test_size=0.05, random_state=42)
print((x_train.shape[0], x_test.shape[0]))

print(f'Features extracted: {x_train.shape[1]}')

(1345, 71)
Features extracted: 58


In [10]:
kfold = StratifiedKFold(n_splits=5, random_state=42,shuffle=True)

In [12]:
parameters={'max_features':np.arange(0,59,1)}

rf = RandomForestClassifier()

rfGrid = GridSearchCV(rf,param_grid=parameters, cv=kfold, scoring="accuracy",verbose=3,n_jobs=-1)
rfGrid.fit(x_train, y_train)
y_pred_rf= rfGrid.predict(x_test)

print(accuracy_score(y_test, y_pred_rf))



print("Best Params ",rfGrid.best_params_)
print("Best Score ",rfGrid.best_score_)


Fitting 5 folds for each of 59 candidates, totalling 295 fits
[CV] max_features=0 ..................................................
[CV] ........................ max_features=0, score=nan, total=   0.0s
[CV] max_features=0 ..................................................
[CV] ........................ max_features=0, score=nan, total=   0.0s
[CV] max_features=0 ..................................................
[CV] ........................ max_features=0, score=nan, total=   0.0s
[CV] max_features=0 ..................................................
[CV] ........................ max_features=0, score=nan, total=   0.0s
[CV] max_features=0 ..................................................
[CV] ........................ max_features=0, score=nan, total=   0.0s


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\dbda\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
C:\Users\dbda\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: max_features must be in (0, n_features]

  FitFailedWarning)
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
C:\Users\dbda\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:536: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parame

[CV] max_features=1 ..................................................
[CV] ...................... max_features=1, score=0.558, total=   0.2s
[CV] max_features=1 ..................................................
[CV] ...................... max_features=1, score=0.550, total=   0.2s
[CV] max_features=1 ..................................................
[CV] ...................... max_features=1, score=0.509, total=   0.2s
[CV] max_features=1 ..................................................
[CV] ...................... max_features=1, score=0.528, total=   0.2s
[CV] max_features=1 ..................................................
[CV] ...................... max_features=1, score=0.532, total=   0.2s
[CV] max_features=2 ..................................................
[CV] ...................... max_features=2, score=0.606, total=   0.2s
[CV] max_features=2 ..................................................
[CV] ...................... max_features=2, score=0.610, total=   0.2s
[CV] m

[CV] ..................... max_features=12, score=0.625, total=   0.7s
[CV] max_features=12 .................................................
[CV] ..................... max_features=12, score=0.550, total=   0.8s
[CV] max_features=13 .................................................
[CV] ..................... max_features=13, score=0.602, total=   0.7s
[CV] max_features=13 .................................................
[CV] ..................... max_features=13, score=0.584, total=   0.7s
[CV] max_features=13 .................................................
[CV] ..................... max_features=13, score=0.572, total=   0.8s
[CV] max_features=13 .................................................
[CV] ..................... max_features=13, score=0.576, total=   0.8s
[CV] max_features=13 .................................................
[CV] ..................... max_features=13, score=0.558, total=   0.7s
[CV] max_features=14 .................................................
[CV] .

[CV] ..................... max_features=24, score=0.599, total=   1.3s
[CV] max_features=24 .................................................
[CV] ..................... max_features=24, score=0.550, total=   1.2s
[CV] max_features=24 .................................................
[CV] ..................... max_features=24, score=0.602, total=   1.3s
[CV] max_features=24 .................................................
[CV] ..................... max_features=24, score=0.546, total=   1.3s
[CV] max_features=25 .................................................
[CV] ..................... max_features=25, score=0.580, total=   1.3s
[CV] max_features=25 .................................................
[CV] ..................... max_features=25, score=0.572, total=   1.5s
[CV] max_features=25 .................................................
[CV] ..................... max_features=25, score=0.602, total=   1.2s
[CV] max_features=25 .................................................
[CV] .

[CV] ..................... max_features=35, score=0.569, total=   1.8s
[CV] max_features=36 .................................................
[CV] ..................... max_features=36, score=0.576, total=   1.8s
[CV] max_features=36 .................................................
[CV] ..................... max_features=36, score=0.558, total=   1.9s
[CV] max_features=36 .................................................
[CV] ..................... max_features=36, score=0.565, total=   1.8s
[CV] max_features=36 .................................................
[CV] ..................... max_features=36, score=0.599, total=   1.9s
[CV] max_features=36 .................................................
[CV] ..................... max_features=36, score=0.591, total=   1.8s
[CV] max_features=37 .................................................
[CV] ..................... max_features=37, score=0.565, total=   2.0s
[CV] max_features=37 .................................................
[CV] .

[CV] ..................... max_features=47, score=0.558, total=   2.4s
[CV] max_features=47 .................................................
[CV] ..................... max_features=47, score=0.572, total=   2.4s
[CV] max_features=47 .................................................
[CV] ..................... max_features=47, score=0.565, total=   2.5s
[CV] max_features=48 .................................................
[CV] ..................... max_features=48, score=0.554, total=   2.5s
[CV] max_features=48 .................................................
[CV] ..................... max_features=48, score=0.546, total=   2.5s
[CV] max_features=48 .................................................
[CV] ..................... max_features=48, score=0.576, total=   2.5s
[CV] max_features=48 .................................................
[CV] ..................... max_features=48, score=0.595, total=   2.6s
[CV] max_features=48 .................................................
[CV] .

[Parallel(n_jobs=1)]: Done 295 out of 295 | elapsed:  7.6min finished


0.704225352112676
Best Params  {'max_features': 4}
Best Score  0.5962825278810409


In [68]:
#################### Prediction on unseen data
def load_test_data():
    x_test,y_test=[],[]
    
    file="F:\\speech_project\\UnSeenSet\\03-01-07-01-01-02-15.wav"
    emotion=emotions[file.split("-")[2]]
    feature=extract_feature(file, mfcc=True, chroma=True,spectral_centroid=True,spectral_bandwidth=True,
                            spectral_rolloff=True,spectral_contrast=True,rms=True,spectral_flatness=True)
    
    x_test.append(feature)
    y_test.append(emotion)
    print(x_test)
    return x_test,y_test

In [69]:
X_test_data,y_test_data=load_test_data()
print(y_test_data)

[array([-5.76542969e+02,  6.80835495e+01, -2.08600235e+01,  1.93035564e+01,
        1.35577583e+00, -1.16599786e+00, -6.15480709e+00, -5.99488688e+00,
        1.69360983e+00,  2.48043895e+00, -1.89693403e+00, -8.06386828e-01,
        1.91894567e+00,  6.07720947e+00, -9.03752983e-01, -9.56951082e-01,
        7.98137605e-01, -4.33641291e+00,  4.76005189e-02,  1.36326599e+00,
       -2.66334414e-01,  9.42056358e-01,  1.03757173e-01,  3.81496608e-01,
        4.33735341e-01,  1.11797643e+00, -5.18218040e-01,  2.82531047e+00,
        9.52483118e-02,  1.69378901e+00, -2.51021296e-01,  2.14043283e+00,
        3.39507866e+00,  3.61230636e+00,  2.16703892e+00,  2.10190296e+00,
        3.72034073e-01,  1.80648112e+00, -3.42183448e-02, -1.00127256e+00,
        5.94972134e-01,  5.97770333e-01,  5.77731550e-01,  6.29865050e-01,
        7.22877383e-01,  7.31866360e-01,  6.78925991e-01,  7.01202929e-01,
        7.34922707e-01,  7.51407385e-01,  7.65149891e-01,  6.68664038e-01,
        2.17363899e+03, 

In [ ]:
y_test_pred=rfGrid.predict(X_test_data)
print("Unseen data:: ", y_test_pred)

In [66]:
print(le.inverse_transform(y_test_pred)[0])

calm


In [ ]:
############# SAVING THE MODEL
import pickle
pickle.dump(rfGrid, open("C:\\Users\\dbda\\Desktop\\project\\RF.pkl", 'wb'))
print("Model Saved!!")

In [70]:
import pickle
loaded_model = pickle.load(open("C:\\Users\\dbda\\Desktop\\project\\RF.pkl", 'rb'))
result = loaded_model.predict(X_test_data)
print(result)

[4]


In [71]:
print(le.inverse_transform(result))

['happy']
